In [1]:
using Distributions, Random
using JLD2
using greedy
using Shared
using NumOpt

In [2]:
Random.seed!(113)

TaskLocalRNG()

<h1>Tasks setup</H1>

In [3]:
const T = 96
const Δt = 15.0 / 60
const base_load_total = rand(Normal(1000.0 , 20.0 ), T)
base_load = [base_load_total ./ 3  base_load_total ./ 3  base_load_total ./ 3]
@show size(base_load)
price = rand(Normal(0.8, 0.2), T)
clamp!(price, 0.4, 1.1)
;

size(base_load) = (96, 3)


In [4]:
extrema(base_load_total)

(959.1492756168113, 1052.0311759279086)

In [5]:
P_max = 6000.0
pi_max = 8e-2
;

In [6]:
K = 400
tasks = gen_tasks(T, K)
env = Environment(T, Δt, P_max, pi_max, base_load, price, tasks)
;

<h1> Numerical optimization </h1>

In [7]:
# model, solve_time= solve!(env; optimizer=:Cbc, opt_options=Dict("logLevel"=>0, "ratioGap"=>10e-3));
model, solve_time= solve!(env; optimizer=:Gurobi);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-15


┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP C:\Users\lenovo\.julia\packages\JuMP\7rBNn\src\operators.jl:282


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9700 CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1072 rows, 38400 columns and 307200 nonzeros
Model fingerprint: 0x8aa2646d
Variable types: 0 continuous, 38400 integer (38400 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [4e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 5e+03]
Found heuristic solution: objective 17049.978075
Presolve removed 74 rows and 13214 columns
Presolve time: 0.17s
Presolved: 998 rows, 25186 columns, 174277 nonzeros
Variable types: 0 continuous, 25186 integer (25154 binary)
Found heuristic solution: objective 16708.291251

Root relaxation: objective 1.435039e+04, 2306 iterations, 0.07 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [8]:
@show solve_time

solve_time = 216.17300009727478


216.17300009727478

In [9]:
# @show model
x = get_decision_matrix(model)
validate_solution_feasibility(x, env)

👍The solution is feasible!


In [10]:
obj = get_value(model[:obj])

14352.025028387176

In [11]:
get_value(model[:P_total])  |> extrema

(971.36397108604, 3778.6092421129324)

In [12]:
root_dir = @__DIR__ 
jldsave(joinpath(root_dir, "data/tasks113.jld2"); env, x, obj)

<h1> Greedy optimization </h1>

In [13]:
result, GY_solve_time = run_greedy(env)

([0 1 … 0 1; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 0.34800004959106445)

In [14]:
get_value_greedy(result, env)

14385.4158111862

In [15]:
validate_solution_feasibility(result, env)

👍The solution is feasible!


In [16]:
power = compute_power(result, env)
P_total = sum(power, dims=2)
err_power = filter(x -> x > env.P_max, P_total)
imb = compute_imbalance(power)
err_imb = filter(x -> x > env.pi_max, imb)
@show err_power
@show err_imb
;

err_power = Float64[]
err_imb = Float64[]


In [17]:
exp_h = env.tasks.h
gdy_h = sum(result, dims=1) |> vec
err = exp_h - gdy_h
err_h = filter(x -> x != 0, err)
@show err_h
;

err_h = Int64[]
